# The Project #
1. This is a project with minimal scaffolding. Expect to use the the discussion forums to gain insights! It’s not cheating to ask others for opinions or perspectives!
2. Be inquisitive, try out new things.
3. Use the previous modules for insights into how to complete the functions! You'll have to combine Pillow, OpenCV, and Pytesseract
4. There are hints provided in Coursera, feel free to explore the hints if needed. Each hint provide progressively more details on how to solve the issue. This project is intended to be comprehensive and difficult if you do it without the hints.

### The Assignment ###
Take a [ZIP file](https://en.wikipedia.org/wiki/Zip_(file_format)) of images and process them, using a [library built into python](https://docs.python.org/3/library/zipfile.html) that you need to learn how to use. A ZIP file takes several different files and compresses them, thus saving space, into one single file. The files in the ZIP file we provide are newspaper images (like you saw in week 3). Your task is to write python code which allows one to search through the images looking for the occurrences of keywords and faces. E.g. if you search for "pizza" it will return a contact sheet of all of the faces which were located on the newspaper page which mentions "pizza". This will test your ability to learn a new ([library](https://docs.python.org/3/library/zipfile.html)), your ability to use OpenCV to detect faces, your ability to use tesseract to do optical character recognition, and your ability to use PIL to composite images together into contact sheets.

Each page of the newspapers is saved as a single PNG image in a file called [images.zip](./readonly/images.zip). These newspapers are in english, and contain a variety of stories, advertisements and images. Note: This file is fairly large (~200 MB) and may take some time to work with, I would encourage you to use [small_img.zip](./readonly/small_img.zip) for testing.

Here's an example of the output expected. Using the [small_img.zip](./readonly/small_img.zip) file, if I search for the string "Christopher" I should see the following image:
![Christopher Search](./readonly/small_project.png)
If I were to use the [images.zip](./readonly/images.zip) file and search for "Mark" I should see the following image (note that there are times when there are no faces on a page, but a word is found!):
![Mark Search](./readonly/large_project.png)

Note: That big file can take some time to process - for me it took nearly ten minutes! Use the small one for testing.

In [ ]:
import zipfile

from PIL import Image
from IPython.display import display
import pytesseract
import cv2 as cv
import numpy as np
import os

face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')

with zipfile.ZipFile("readonly/images.zip", "r") as zip_ref:
    zip_ref.extractall("extracted/")
    
def scan_text(page):
    page_img = Image.open(page)
    text_str = pytesseract.image_to_string(page_img).lower()
    return text_str.split()

def find_faces(page):
    cv_page = cv.imread(page)
    gray_page = cv.cvtColor(cv_page, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_page, scaleFactor=1.3, minNeighbors=5, minSize=(25,25))
    contact_sheet = None    
    if len(faces) > 0:
        PIL_page = Image.open(page)
        contact_sheet = Image.new(PIL_page.mode, (100*5, 100*(len(faces)//5+1)))
        c_x = 0
        c_y = 0
        for x, y, w, h in faces:
            face = PIL_page.crop((x, y, x+w, y+h))
            contact_sheet.paste(face.resize((100,100), Image.ANTIALIAS), (c_x,c_y))
            if c_x == 400:
                c_x = 0
                c_y += 100
            else:
                c_x += 100
    return contact_sheet

def search(tag):
    print("Search tag: " + tag)
    for page in os.listdir("extracted/"):
        page_path = "extracted/" + page
        if tag.lower() in scan_text(page_path):
            print("Results found in file " + page)
            if find_faces(page_path) is None:
                print("But there were no faces in that file!")
            else:
                display(find_faces(page_path))
                
search("Christopher")
search("Mark")
